# PS6 Dijkstra

## Shortest path using Dijkstra's algorithm code for MIT 6.006 Fall 2011 PS6

The code distribution contains the following files:
  * dijkstra.py - pset (stripped) version with unimplemented Dijkstra's algorithm
  * dijkstra_test.py - tester used for grading
  * nhpn.py - loader for parsing data from NHPN files
  * priority_queue.py - heap-based priority queue
  * data/nhpn.lnk - link data from NHPN database
  * data/nhpn.nod - node data from NHPN database
  * data/{datadict, format}.txt - data format description  
  * tests/*.in - bi-directional dijkstra test inputs
  * tests/*.gold - outputs that we believe to be correct for the dijkstra test 
      inputs

## USAGE

dijkstra.py's behavior can be tweaked using the TRACE environment variable. If 
TRACE=kml, two kml files path_flat.kml and path_curved.kml will be created 
representing the shortest path from the source to the destination. The files can
be viewed on Google Map. If not, a text description of the path is output to the
stdout. 

Command-line example:
    TRACE=kml python dijkstra.py < tests/0boston_berkeley.in


## DEPENDENCIES

{dijkstra, priority_queue, nhpn}.py have been tested on Python 2.7 and PyPy 1.6.

In [2]:
class PriorityQueue:
    """Min-heap-based priority queue, using 1-based indexing. Adapted from CLRS.
    
    Augmented to include a map of keys to their indices in the heap so that
    key lookup is constant time and decrease_key(key) is O(log n) time.
    """
    
    def __init__(self):
        """Initializes the priority queue."""
        self.heap = [None] # To make the index 1-based.
        self.key_index = {} # key to index mapping.
    
    def __len__(self):
        return len(self.heap) - 1
       
    def __getitem__(self, i):
        return self.heap[i]

    def __setitem__(self, i, key):
        self.heap[i] = key

    def decrease_key(self, key):
        """Decreases the value of the key if it is in the priority queue and 
        maintains the heap property."""
        index = self.key_index[key]
        if index:
            self._decrease_key(index, key)
    
    def insert(self, key):
        """Inserts a key into the priority queue."""
        self.heap.append(key)
        self.key_index[key] = len(self)
        self._decrease_key(len(self), key)

    def extract_min(self):
        """Removes and returns the minimum key."""
        if len(self) < 1:
            return None
        self._swap(1, len(self))
        min = self.heap.pop()
        del self.key_index[min]
        self._min_heapify(1)
        return min
    
    def _decrease_key(self, i, key):
        """Decreases key at a give index.
        
        Args:
            i: index of the key.
            key: key with decreased value.
        """
        while i > 1:
            parent = i // 2
            if self[parent] > key:
                self._swap(i, parent)
                i = parent
            else:
                break
            
    def _min_heapify(self, i):
        """Restores the heap property from index i downwards."""
        l = 2 * i
        r = 2 * i + 1
        smallest = i
        if l <= len(self) and self[l] < self[i]:
            smallest = l
        if r <= len(self) and self[r] < self[smallest]:
            smallest = r
        if smallest != i:
            self._swap(i, smallest)
            self._min_heapify(smallest)

    def _swap(self, i, j):
        # Swaps the key at indices i and j and updates the key to index map.
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]
        self.key_index[self.heap[i]], self.key_index[self.heap[j]] = i, j

    def check_ri(self):
        heap = self.heap
        i = 1
        while i <= (len(heap) - 1) // 2:
            l = i * 2
            if heap[i] > heap[l]:
                raise ValueError('Left child is smaller than parent.')
            r = i * 2 + 1
            if r < len(heap) and heap[i] > heap[r]:
                raise ValueError('Right child is smaller than parent.')
            i += 1
            
        for key, index in self.key_index.items():
            if self.heap[index] is not key:
                raise ValueError('Key index mapping is wrong.')

In [3]:
class Vertex(object):
    def __init__(self):
        self.predecessor = None
        self.colour = black
        self.distance = 1000
        self.neighbours = {}
    
    def __gt__(self, value):
        return self.distance > other.distance
    
    def __lt__(self, value):
        return self.distance < other.distance
    
    def _eq__(self, value):
        return self.distance == other.distance

In [4]:
Q = PriorityQueue()

Q.insert(5)
Q.insert(4)
Q.insert(3)
Q.insert(2)
Q.insert(1)

while Q:
    u = Q.extract_min()
    print(u)

1
2
3
4
5


In [5]:
# File:   nhpn.py
# Author: Punyashloka Biswal
# Date:   04/09/2007
# Modified: Michael Lieberman
# Date:   04/03/2008

"""Defines Python classes corresponding to nodes and edges in the
National Highway Planning Network (NHPN) database. Also provides a
way to load such objects from files."""

class Node:
    """An NHPN geographical node."""

    def __init__(self, longitude, latitude, state, description):
        """Creates an instance of a node from its longitude, latitude,
        two-character state code, and possibly a description.
        string."""
        self.longitude = longitude
        self.latitude = latitude
        self.state = state
        self.description = description

    def __repr__(self):
        """Convert to string for printing."""
        return "Node(%s, %s, '%s', '%s')" % (self.longitude, self.latitude,
                                             self.state, self.description)

In [6]:
class Link:
    """A bi-directional edge linking two NHPN nodes."""
    
    def __init__(self, begin, end, description):
        """Creates a link given its beginning and end (which must be nodes) and
        possibly a description string."""
        self.begin = begin
        self.end = end
        self.description = description

    def __repr__(self):
        """Converts to string for printing."""
        return "Link(%s, %s, '%s')" % (self.begin, self.end, self.description)

In [7]:
class Loader:
    """An instance of Loader can be used to access NHPN nodes and links as
    Python objects."""

    def __init__(self, nodesource='data/nhpn.nod', linksource='data/nhpn.lnk'):
        """Load node and link objects from corresponding files."""
        nodeForFeatureID = {}   # FeatureID -> node mapping

        # Load nodes and add to feature table
        self._nodes = []
        try:
            nodefile = open(nodesource, 'r')
            for line in nodefile:
                featureId = int(line[23:33])
                longitude = int(line[33:43])
                latitude = int(line[43:53])
                state = line[53:55].strip()
                description = line[55:88].strip()
                
                node = Node(longitude, latitude, state, description)
                nodeForFeatureID[featureId] = node
                self._nodes.append(node)
        finally:
            nodefile.close()

        # Load links
        links = []
        try:
            linkfile = open(linksource, 'r')
            for line in linkfile:
                begin = nodeForFeatureID[int(line[33:43])]
                end = nodeForFeatureID[int(line[43:53])]
                description = line[53:88].strip()
                links.append(Link(begin, end, description))
        finally:
            linkfile.close()
        self._links = links

    def nodes(self):
        """List of all NHPN nodes."""
        return self._nodes

    def links(self):
        """List of all NHPN links."""
        return self._links

In [8]:
raw_network = Loader()

print("Nodes: %s, Links: %s" % (len(raw_network._nodes), len(raw_network._links)))


Nodes: 90415, Links: 125302


In [9]:
#!usr/bin/env python

import sys
import os
import time
from math import *
#from nhpn import *
#from priority_queue import *

def distance(node1, node2):
    """Returns the distance between node1 and node2, ignoring the Earth's 
    curvature.
    """
    latitude_diff = node1.latitude - node2.latitude
    longitude_diff = node1.longitude - node2.longitude
    return (latitude_diff**2 + longitude_diff**2)**.5

def distance_curved(node1, node2):
    """Returns the distance between node1 and node2, including the Earth's 
    curvature.
    """
    A = node1.latitude * pi / 10**6 / 180
    B = node1.longitude * pi / 10**6 / 180
    C = node2.latitude * pi / 10**6 / 180
    D = node2.longitude * pi / 10**6 / 180
    return acos(sin(A) * sin(C) + cos(A) * cos(C) * cos(B - D))

In [10]:
class NodeDistancePair(object):
    """Wraps a node and its distance representing it in the priority queue."""
    
    def __init__(self, node, distance, predecessor=None):
        """Creates a NodeDistancePair to be used as a key in the priority queue.
        """
        self.node = node
        self.distance = distance
        self.predecessor = predecessor

    def __str__(self):
        return str(self.node) + str(self.distance)
    
    def __lt__(self, other):
        # :nodoc: Delegate comparison to distance.
        return (self.distance < other.distance or 
                (self.distance == other.distance and 
                 id(self.node) < id(other.node)))
    
    def __le__(self, other):
        # :nodoc: Delegate comparison to distance.
        return (self.distance < other.distance or
                (self.distance == other.distance and 
                 id(self.node) <= id(other.node)))
                
    def __gt__(self, other):
        # :nodoc: Delegate comparison to distance.
        return (self.distance > other.distance or
                (self.distance == other.distance and 
                 id(self.node) > id(other.node)))
    
    def __ge__(self, other):
        # :nodoc: Delegate comparison to distance.
        return (self.distance > other.distance or
                (self.distance == other.distance and 
                 id(self.node) >= id(other.node)))

In [11]:
class Network(object):
    """The National Highway Planning network."""
    def __init__(self):
        """Creates a network with nodes, links and an edge set."""
        self.nodes, self.links = self._load_data()
        self._create_adjacency_lists()
        self.edge_set = self._create_edge_set()
    
    def __str__(self):
        """String representation of the network size."""
        num_nodes = len(self.nodes)
        num_edges = 0
        for node in self.nodes:
            num_edges += len(node.adj)
        return "Graph size: %d nodes, %d edges" % (num_nodes, num_edges)
        
    def verify_path(self, path, source, destination):
        """Verifies that path is a valid path from source to destination.
        
        Returns:
            True if the path is valid such that it uses only edges in the edge
            set.
            
        Raises:
            ValueError: if the first node and the last node do not match source
                and destination respectively or if the edge in not the the edge
                set.
        """
        if source != path[0]:
            raise ValueError('First node on a path is different form the \
                              source.')
        if destination != path[-1]:
            raise ValueError('Last node on a path is different form the \
                              destination.')
        for i in range(len(path) - 1):
            if (path[i], path[i+1]) not in self.edge_set and \
                (path[i+1], path[i]) not in self.edge_set:
                raise ValueError('Adjacent nodes in path have no edge between \
                                  them')
        return True

    def node_by_name(self, city, state):
        """Returns the first node that matches specified location.
        
        Args:
            city: the description of the node should include city.
            state: the state of the node should match state.
        
        Returns:
            The node if it exists, or None otherwise.
        """
    
        for node in self.nodes:
            if node.state == state:
                if city in node.description:
                    return node
        return None
    
    def _load_data(self):
        loader = Loader()
        lnodes = loader.nodes()
        llinks = loader.links()
        return lnodes, llinks
    
    def _create_adjacency_lists(self):
        # Use an adjacency list representation, by putting all vertices
        # adjacent to node in node.adj.
        for node in self.nodes:
            node.adj = []
        for link in self.links:
            link.begin.adj.append(link.end)
            link.end.adj.append(link.begin)
            
    def _create_edge_set(self):
        # Puts edges in a set for faster lookup.
        edge_set = set()
        for link in self.links:
            edge_set.add((link.begin, link.end))
        return edge_set
            

In [12]:
network = Network()
print(network)
print(network.nodes[1].adj)

Graph size: 90415 nodes, 250604 edges
[Node(-87820450, 35000019, '', ''), Node(-87761719, 35123020, '', '')]


In [15]:
class PathResult(object):
    """An object containing the results of a path found by PathFinder."""
    
    def __init__(self, path, num_visited, weight, network, time):
        """Creates a PathResult.
        
        Args:
            path: a list of nodes in the path.
            num_visited: number of nodes visited during path finding.
            weight: function to compute the weight of an edge (u, v).
            network: the network on which the path is found.
            time: time used to find the path.
        """
        self.network = network
        self.path = path
        self.num_visited = num_visited
        self.total_weight = self._total_weight(weight)
        self.time = time
        
    def to_kml(self):
        """Returns the path in kml format."""
        
        kml = ["""<?xml version="1.0" encoding="utf-8"?>
<kml xmlns="http://earth.google.com/kml/2.1">
  <Document>
    <Placemark>
      <LineString>
        <extrude>1</extrude>
        <tessellate>1</tessellate>
        <coordinates>
"""]
        kml.append(''.join("%f,%f\n" % (node.longitude/1000000., 
                                        node.latitude/1000000.) 
                           for node in self.path))
        kml.append("""</coordinates>
      </LineString>
    </Placemark>
  </Document>
</kml>
""")
        return ''.join(kml)
    
    def to_lines(self):
        """Returns a list of lines representing the results."""
        source = self.path[0]
        dest = self.path[-1]
        list = ["Path: %s, %s -> %s, %s" %  (source.description, \
                    source.state, dest.description, dest.state)]
        list.append(str(self.network))
        list.append("Nodes visited: %d" % self.num_visited)
        list.append("Path length: %.4f" % self.total_weight)
        list.append("Number of roads: %d" % (len(self.path) - 1))
        list.append("Time used in seconds: %.3f" % self.time)
        return list
    
    def sol_to_lines(self):
        return ["Path length: %.4f" % self.total_weight]
    
    def to_file(self, file):
        """Outputs to an output stream."""
        for line in self.to_lines():
            file.write(line)
            file.write("\n")
    
    def sol_to_file(self, file):
        """Outputs solution to output stream."""
        for line in self.sol_to_lines():
            file.write(line)
            file.write("\n")
                    
    def _total_weight(self, weight):
        """Computes the sum of weights along a path.
        
        Args:
            weight: function to compute the weight of an edge (u, v).
        """
        sum = 0
        for i in range(len(self.path) - 1):
            sum += weight(self.path[i], self.path[i + 1])
        return sum

# Command-line controller.
if __name__ == '__main__':
    network = Network()
    if os.environ.get('TRACE') == 'kml':
        pf = PathFinder.from_file(sys.stdin, network)
        with open('path_flat.kml', 'w') as file:
            r = pf.shortest_path(distance)
            r and file.write(r.to_kml())
        with open('path_curved.kml', 'w') as file:
            r = pf.shortest_path(distance_curved)
            r and file.write(r.to_kml())
    else:
        pf = PathFinder.from_file(sys.stdin, network)
        r = pf.shortest_path(distance)
        if r:
            if os.environ.get('TRACE') == 'sol':
                r.sol_to_file(sys.stdout)
            else:
                r.to_file(sys.stdout)
        else:
            print('No path is found.')
    

IndexError: list index out of range

In [82]:
class PathFinder(object):
    """Finds a shortest path from the source to the destination in the network.
    """
    def __init__(self, network, source, destination):
        """Creates a PathFinder for the network with source and destination.
        
        Args:
            network: the network on which paths should be found.
            source: source of the path.
            destination: destination of the path.
        """    
        self.network = network
        self.source = source
        self.destination = destination
        
    def shortest_path(self, weight):
        """Returns a PathResult for the shortest path from source to destination. 
        
        Args: 
            weight: weight function to compute edge weights.
            
        Returns:
            PathResult for the shortest path or None if path is empty.
        """
        start_time = time.clock()
        
        path, num_visited = self.dijkstra(weight, self.network.nodes, 
                                          self.source, self.destination)
            
        time_used = round(time.clock() - start_time, 3)
        if path:
            if self.network.verify_path(path, self.source, self.destination):
                return PathResult(path, num_visited, weight, self.network, 
                                  time_used)
        else:
            return None

    @staticmethod
    def from_file(file, network):
        """Creates a PathFinder object with source and destination read from 
        file.
        
        Args:
            file: file containing source and destination.
            network: network in which a shortest path needs to be found.
        
        Returns:
            A PathFinder object.
            
        Raises:
            ValueError: when source or destination is not valid.
        """
        source = destination = None
        for i in range(2):
            command = file.readline().split()
            city = ' '.join(command[1].split('_')).upper()
            node = network.node_by_name(city, command[2].upper())
            if command[0] == 'source':
                source = node
            elif command[0] == 'destination':
                destination = node
                
        if source and destination:
            return PathFinder(network, source, destination)
        else:
            if source is None:
                raise ValueError('Invalid source.')
            if destination is None:
                raise ValueError('Invalid destination.')  

    def dijkstra(self, weight, nodes, source, destination):
        """Performs Dijkstra's algorithm until it finds the shortest
        path from source to destination in the graph with nodes and edges.
        Assumes that all weights are non-negative.
    
        At the end of the algorithm:
        - node.visited is True if the node is visited, False otherwise.
        (Note: node is visited if the shortest path to it is computed.)
    
        Args:
            weight: function for calculating the weight of edge (u, v). 
            nodes: list of all nodes in the network.
            source: the source node in the network.
            destination: the destination node in the network.
         
        Returns:
            A tuple: (the path as a list of nodes from source to destination, 
                      the number of visited nodes)
        """
        # Initialise distances to infinity
        # Initialise starting node distance to zero
        # Add all nodes to prirority queue
        INFINITE_DISTANCE = 1234567890
        Q = PriorityQueue()
        nodedistancemap = {} # easily retrieve node-distance pair from node
        for node in nodes:
            if node == source:
                print("Found source node: " + str(node))
                distance = 0
            else:
                if node == destination:
                    print("Found destination node: " +str(node))
                distance = INFINITE_DISTANCE
            ndp = NodeDistancePair(node, distance)
            Q.insert(ndp)
            nodedistancemap[node] = ndp
        
        while Q: # While there are nodes in the Q
            #Q.check_ri()
            ndp = Q.extract_min()
            #print("Process node: " + str(ndp))
            if ndp.distance == INFINITE_DISTANCE:
                print("Relaxing node with infinite distance" + str(ndp))
                break
            for v in ndp.node.adj:
                vdp = nodedistancemap[v]
                self.relax(ndp, vdp, weight) # Relax the outgoing edges
                if vdp in Q.key_index:
                    Q.decrease_key(vdp)
            if ndp.node == destination: # optimisation
                print("Processed destination node: " + str(ndp))
                break
        
        # Retrieve the shortest path to the destination
        path = []
        node_on_path = nodedistancemap[destination]
        distance = node_on_path.distance
        while node_on_path:
            #print(node_on_path)
            path.insert(0, node_on_path.node)
            node_on_path = node_on_path.predecessor
        
        print(path)
        print(distance)
        
        return (path, distance) 
    
    def relax(self, u, v, weight):
        #print(str(u))
        #print(str(v))
        #print(weight(u.node, v.node))
        if (v.distance >= u.distance + weight(u.node, v.node)):
            #print("Relaxing: %s -> %s weight: %d" % (str(u), str(v), weight(u.node, v.node)))
            v.distance = u.distance + weight(u.node, v.node)
            v.predecessor = u
     

In [83]:
in_filename = "tests/0boston_berkeley.in"
in_filename = "tests/1pasadena_cambridge.in"
in_filename = "tests/5berkeley_boston.in"
with open(in_filename) as in_file:
    pf = PathFinder.from_file(in_file, network)
    result = pf.shortest_path(distance)
    network.verify_path(result.path, pf.source, pf.destination)
    out_lines = result.sol_to_lines()

Found source node: Node(-122271637, 37871849, 'CA', 'BERKELEY')
Found destination node: Node(-71037895, 42373821, 'MA', 'BOSTON EAST BOST')
Processed destination node: Node(-71037895, 42373821, 'MA', 'BOSTON EAST BOST')54161784.73631543
[Node(-122271637, 37871849, 'CA', 'BERKELEY'), Node(-122269737, 37854633, '', ''), Node(-122258217, 37856506, '', ''), Node(-122227600, 37849209, '', ''), Node(-122205940, 37821926, '', ''), Node(-122196274, 37810799, '', ''), Node(-122177246, 37783970, '', ''), Node(-122172317, 37781197, '', ''), Node(-122121452, 37708496, '', ''), Node(-122097435, 37690311, 'CA', 'CASTRO VALLEY W'), Node(-122093285, 37690010, '', ''), Node(-122072678, 37691135, '', ''), Node(-122061401, 37692760, '', ''), Node(-122056664, 37693741, '', ''), Node(-122014816, 37702618, '', ''), Node(-122000008, 37703220, '', ''), Node(-121954712, 37698315, '', ''), Node(-121921539, 37700748, 'CA', 'CP PARKS W  T'), Node(-121817017, 37700909, '', ''), Node(-121772888, 37700855, '', ''), 

In [15]:
a = []
a.insert(0, "foo")
a.insert(0, "bar")
print(a)

['bar', 'foo']


In [46]:
#!/usr/bin/env python

import unittest
import sys
import os
import glob
import re

#if os.environ.get('SOLUTION'):
#    from dijkstra_full import *
#else:
#    from dijkstra import *

class DijkstraTest(unittest.TestCase):
    def setUp(self):
        #dir = os.path.dirname(__file__)
        #self._in_files = glob.glob(os.path.join(dir, 'tests', '*.in'))
        self._in_files = glob.glob(os.path.join('tests', '*.in'))
        self._in_files.sort()
    
    def _cmp_files(self, file, lines):
        """Compares the result with the gold result.
        
        Args:
            file: gold result file.
            lines: result in lines for testing.
            
        Returns:
            True if the result is same as the gold result; False otherwise
        """
        for line in lines:
            if line != file.readline().strip():
                return False
        return file.readline() == ''

    def testCorrectness(self):
        print('Testing correctness:')
        for in_filename in self._in_files:
            test_name = os.path.basename(in_filename)
            sys.stdout.write('Testing {0} ...... '.format(test_name))
            sys.stdout.flush()
            network = Network()
            with open(in_filename) as in_file:
                pf = PathFinder.from_file(in_file, network)
                result = pf.shortest_path(distance)
                network.verify_path(result.path, pf.source, pf.destination)
                out_lines = result.sol_to_lines()
                gold_filename = re.sub('\.in$', '.gold', in_filename)
                with open(gold_filename) as gold_file:
                    same = self._cmp_files(gold_file, out_lines)
                    if same:
                        sys.stdout.write('OK\n')
                    else: 
                        sys.stdout.write('Failed\n')
                    self.assertTrue(same)

#if __name__ == '__main__':
print("Hello")
test = DijkstraTest()
test.setUp()
test.testCorrectness()

Hello
Testing correctness:
Testing 0boston_berkeley.in ...... Found source node: Node(-71037895, 42373821, 'MA', 'BOSTON EAST BOST')
Processed destination node
[Node(-122271637, 37871849, 'CA', 'BERKELEY')]
12345678


ValueError: First node on a path is different form the                               source.

# Lessons learnt or learned
2017/09/20

* I made assumptions about how the PriorityQueue worked - I assumed that I could change the priority of a key and the Q would automatically re-order it.  Instead I needed to call decrease key.  Not sure where this assumption came from - I was clear on the concept of the Q, but not the API contract of this Q.
* It wasn't obvious that the PriorityQueue wasn't working as expected due to the test cases producing output that looked like they were coming out in the correct order.  It was only when I created a new test case by reversing one of the existing ones that I noticed the Q not working.  Adding "check_ri" validated the theory that I was breaking the queue.
* I would have no way to know my algortihm is correct if I didn't have the existing .gold expected outputs.  How would you create test cases for a SSSP algorithm?
* Jupyter notebooks are slow with lots of output (at least in Firefox) and it is cumbersome to work with large cells.  Splitting the original files into smaller cells helped make it easier navigate.
* My debugging approach still involved a lot of adding in print statements!  What other approaches could I have taken?
* It would be nice to rely on external files in a notebook
* I have only a qualititive assessment of how efficient my algorithm is (i.e. run it and see how it long it takes on the test cases).  I suppose I can also compare it to my experience with car SatNavs and it seems similar (i.e. 10 seconds ish).  I have no assessment of how well my implementation scales (i.e. if it scales according to theory).  And don't forget space complexity.
